# 1 - Installs and imports

In [ ]:
!pip install --upgrade pip
!pip install sentencepiece
!pip install transformers

     |████████████████████████████████| 1.6MB 18.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: /usr/include/python3.7m
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
     |████████████████████████████████| 1.2 MB 21.1 MB/s 
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  distutils: /usr/local/lib/python3.7/dist-packages
  sysconfig: /usr/lib/python3.7/site-packages
  distutils: /usr/local/include/python3.7/sentencepiece
  sysconfig: /usr/include/python3.7m/sentencepiece
  distutils: /usr/local/bin
  sysconfig: /usr/bin
  distutils: /

In [2]:
#from transformers import AutoTokenizer, AutoModel, TFAutoModel, AutoConfig
#from transformers import AutoModelForSequenceClassification
#from transformers import TFAutoModelForSequenceClassification
#from transformers import pipeline
import numpy as np
from scipy.spatial.distance import cosine
from collections import defaultdict 
import urllib
import numpy as np
from scipy.special import softmax
from sklearn.metrics import classification_report

# 2 - Fetch XLM-T model

In [ ]:
MODEL =  "cardiffnlp/twitter-xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModel.from_pretrained(MODEL)

Some weights of XLMRobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Use Cases

## 1 - Compute Tweet Similarity

In [ ]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def get_embedding(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt')
    features = model(**encoded_input)
    features = features[0].detach().numpy() 
    features_mean = np.mean(features[0], axis=0) 
    return features_mean

query = "Acabo de pedir pollo frito 🐣" #spanish

In [ ]:
tweets = ["We had a great time! ⚽️", # english
          "We hebben een geweldige tijd gehad! ⛩", # dutch
          "Nous avons passé un bon moment! 🎥", # french
          "Ci siamo divertiti! 🍝"] # italian

d = defaultdict(int)
for tweet in tweets:
    sim = 1-cosine(get_embedding(query),get_embedding(tweet))
    d[tweet] = sim

In [ ]:
print('Most similar to: ',query)
print('----------------------------------------')
for idx,x in enumerate(sorted(d.items(), key=lambda x:x[1], reverse=True)):
  print(idx+1,x[0])

Most similar to:  Acabo de pedir pollo frito 🐣
----------------------------------------
1 Ci siamo divertiti! 🍝
2 Nous avons passé un bon moment! 🎥
3 We had a great time! ⚽️
4 We hebben een geweldige tijd gehad! ⛩


## 2 - Simple inference example (with `pipelines`)

In [ ]:
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

In [ ]:
sentiment_task("Huggingface es lo mejor! Awesome library 🤗😎")

[{'label': 'Positive', 'score': 0.9343640804290771}]

# 3 - Experiment on UMSAB

## Fetch dataset (Spanish)

In [3]:
#language = 'spanish'
language = 'german'

files = """test_labels.txt
test_text.txt
train_labels.txt
train_text.txt
val_labels.txt
val_text.txt""".split('\n')

def fetch_data(language, files):
 dataset = defaultdict(list)
 for infile in files:
   thisdata = infile.split('/')[-1].replace('.txt','')
   dataset_url = f"https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/{language}/{infile}"
   print(f'Fetching from {dataset_url}')
   with urllib.request.urlopen(dataset_url) as f:
     for line in f:
       if thisdata.endswith('labels'):
         dataset[thisdata].append(int(line.strip().decode('utf-8')))
       else:
         dataset[thisdata].append(line.strip().decode('utf-8'))
 return dataset     

dataset = fetch_data(language, files)

Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/test_labels.txt
Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/test_text.txt
Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/train_labels.txt
Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/train_text.txt
Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/val_labels.txt
Fetching from https://raw.githubusercontent.com/cardiffnlp/xlm-t/main/data/sentiment/german/val_text.txt


In [4]:
dataset['train_text'][:3],dataset['train_labels'][:3]

(['Normal bin ich ja ned der mensch dwer sich beschwert wegen dem essen aber diese Pizza von Joeys...boah wie ekelhaft',
  'VfB Stuttgart gegen SV Hamburg: Das Warten auf den nächsten Schritt - Stuttgarter Zeitung: DIE WELTVfB Stuttga... http',
  '@user @user @user @user @user @user @user @user fühlt sich schon so spät an 😚😚😚'],
 [0, 1, 2])

In [ ]:
# format data
import pandas as pd

sentiment_map = {
    0: "negative",
    1: "neutral",
    2: "positive"
}

combined_train = [
    {
        "text": line[1][0],
        "original_label": line[1][1],
        "ttlab_label": line[1][1]-1,
        "split": "train",
        "id": line[0],
        "sentiment": sentiment_map[line[1][1]]
    }
    for line
    in enumerate(zip(dataset["train_text"], dataset["train_labels"]))
]
df_train = pd.DataFrame(combined_train)

combined_val = [
    {
        "text": line[1][0],
        "original_label": line[1][1],
        "ttlab_label": line[1][1]-1,
        "split": "dev",
        "id": line[0],
        "sentiment": sentiment_map[line[1][1]]
    }
    for line
    in enumerate(zip(dataset["val_text"], dataset["val_labels"]))
]
df_val = pd.DataFrame(combined_val)

combined_test = [
    {
        "text": line[1][0],
        "original_label": line[1][1],
        "ttlab_label": line[1][1]-1,
        "split": "test",
        "id": line[0],
        "sentiment": sentiment_map[line[1][1]]
    }
    for line
    in enumerate(zip(dataset["test_text"], dataset["test_labels"]))
]
df_test = pd.DataFrame(combined_test)

df = pd.concat([df_train, df_val, df_test])
df.set_index("id", inplace=True)
df["dataset"] = f"umsab/{language}"
df


In [1]:
# export to db
import sqlite3

db_dir = "/home/daniel/data/uni/masterarbeit-sentiment/data/datasets/experiments/de/3sentiment"
db_file = f"{db_dir}/datasets.db"

#con = sqlite3.connect(db_file)
#df.to_sql("dataset", con=con, index=True, index_label="id", if_exists='append')
#con.close()

In [ ]:
# Get combined dataset
import pandas as pd

con = sqlite3.connect(db_file)
df_combined = pd.read_sql("SELECT * FROM dataset", con=con)
con.close()

df_combined

In [3]:
# Preprocess and export combined dataset
from datasets import Dataset, DatasetDict, Split

dataset_combined = DatasetDict()

dataset_combined["train"] = Dataset.from_pandas(
    df_combined[df_combined["split"] == "train"],
    split=Split.TRAIN
)
dataset_combined["test"] = Dataset.from_pandas(
    df_combined[df_combined["split"] == "test"],
    split=Split.TEST
)
dataset_combined["validation"] = Dataset.from_pandas(
    df_combined[df_combined["split"] == "dev"],
    split=Split.VALIDATION
)

def preprocess_combined(sample):
    # text preprocessing
    new_text = []
    for t in sample["text"].split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    sample["text"] = " ".join(new_text)

    # convert to ttlab labels
    # 0 <- -1   negative
    # 1 <-  0    neutral
    # 2 <- +1    positive
    sample["labels"] = sample["ttlab_label"] + 1
    return sample

dataset_combined = dataset_combined.map(preprocess_combined)

dataset_combined = dataset_combined.remove_columns(['original_label', 'ttlab_label', 'sentiment', 'split', '__index_level_0__'])

dataset_combined

  0%|          | 0/1349689 [00:00<?, ?ex/s]

  0%|          | 0/169973 [00:00<?, ?ex/s]

  0%|          | 0/333571 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'dataset', 'labels'],
        num_rows: 1349689
    })
    test: Dataset({
        features: ['id', 'text', 'dataset', 'labels'],
        num_rows: 169973
    })
    validation: Dataset({
        features: ['id', 'text', 'dataset', 'labels'],
        num_rows: 333571
    })
})

In [ ]:
dataset_combined["train"][1500]

In [9]:
combined_dataset_name = "cardiffnlp"
combined_dataset_path = f"/home/daniel/data/uni/masterarbeit-sentiment/data/datasets/experiments/de/3sentiment/{combined_dataset_name}"
combined_dataset_path

'/home/daniel/data/uni/masterarbeit-sentiment/data/datasets/experiments/de/3sentiment/cardiffnlp'

In [10]:
# save preprocessed dataset
dataset_combined.save_to_disk(combined_dataset_path)

## Run full experiment

In [ ]:
# load multilingual sentiment classifier
CUDA = True # set to true if using GPU (Runtime -> Change runtime Type -> GPU)
BATCH_SIZE = 32
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
if CUDA:
  model = model.to('cuda')

In [ ]:
# helper functions
def preprocess(corpus):
  outcorpus = []
  for text in corpus:
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    new_text = " ".join(new_text)
    outcorpus.append(new_text)
  return outcorpus

def predict(text, cuda):
  text = preprocess(text)
  encoded_input = tokenizer(text, return_tensors='pt', padding = True, truncation = True)
  if cuda:
    encoded_input.to('cuda')
    output = model(**encoded_input)
    scores = output[0].detach().cpu().numpy()
  else:
    output = model(**encoded_input)
    scores = output[0].detach().numpy()
  
  scores = softmax(scores, axis=-1)
  return scores

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(dataset['test_text'], batch_size=BATCH_SIZE)
all_preds = []
for idx,batch in enumerate(dl):
  if idx % 10 == 0:
    print('Batch ',idx+1,' of ',len(dl))
  text = preprocess(batch)
  scores = predict(text, CUDA)
  preds = np.argmax(scores, axis=-1)
  all_preds.extend(preds)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Batch  1  of  28
Batch  11  of  28
Batch  21  of  28


In [ ]:
print(classification_report(dataset['test_labels'], all_preds))

              precision    recall  f1-score   support

           0       0.70      0.80      0.75       290
           1       0.62      0.55      0.58       290
           2       0.75      0.74      0.75       290

    accuracy                           0.70       870
   macro avg       0.69      0.70      0.69       870
weighted avg       0.69      0.70      0.69       870

